# **Clasificación**

### En este archivo trataremos con diferentes modelos de clasificación para predecir el ganador del torneo de fútbol de esta temporada 23-24.

Comenzamos importando todas las dependecias necesarias para el correcto funcionamiento del documento.

In [27]:
# Importamos librerías

# Visualización de datos
%matplotlib inline

# Manipulación de datos
import pandas as pd
import numpy as np

# Preparación de los datos
from sklearn.preprocessing import scale, LabelEncoder
from sklearn.model_selection import train_test_split

# Modelos de predicción
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

# Métricas de evaluación
from sklearn.metrics import accuracy_score, classification_report

El siguiente paso es importar los datos con lo que vamos a trabajar.

In [28]:
partidos = pd.read_csv('../../data/partidos_limpio.csv')
partidos.head()

,Season,Round,Day,Date,Results,Home,Country (Home),Points (Home),Score (Home),Score (Away),...,MP_away,Starts_away,Gls_away,Ast_away,G+A_away,G-PK_away,PK_away,PKatt_away,CrdY_away,CrdR_away
0,2023-2024,Round of 16,Tue,2024-02-13,A,RB Leipzig,Germany,88.736698,0,1,...,10.0,110.0,20.0,17.0,37.0,20.0,0.0,1.0,18.0,0.0
1,2023-2024,Round of 16,Tue,2024-02-13,A,FC Copenhagen,Denmark,80.431647,1,3,...,10.0,110.0,28.0,20.0,48.0,25.0,3.0,3.0,10.0,0.0
2,2023-2024,Round of 16,Wed,2024-02-14,H,Paris S-G,France,114.334580,2,0,...,8.0,88.0,8.0,5.0,13.0,8.0,0.0,1.0,18.0,0.0
3,2023-2024,Round of 16,Wed,2024-02-14,H,Lazio,Italy,99.943311,1,0,...,10.0,110.0,18.0,14.0,32.0,16.0,2.0,2.0,13.0,1.0
4,2023-2024,Round of 16,Tue,2024-02-20,D,PSV Eindhoven,The Netherlands,98.784903,1,1,...,10.0,110.0,15.0,12.0,27.0,14.0,1.0,1.0,16.0,0.0


### Preparación de los datos.

Queremos columnas numéricas para nuestra predicción, así que pasaremos todas aquellas columnas categóricas a numéricas usando Label Encoder. Además guardamos estas transformaciones en un diccionario que llamamos 'mapping' para poder posteriormente deshacer los cambios con facilidad. 

La única columna no numérica que no cambiaremos será la columna 'Results' ya que esa será nuestra variable objetivo (y) y podemos dejarla como categórica.

In [29]:
# Columnas a modificar
cols = ['Season', 'Round', 'Day', 'Home', 'Away', 'Country (Home)', 'Country (Away)', 'Venue', 'Referee']

# Inicializamos el label encoder
label_encoder = LabelEncoder()

# Creamos un diccionario para guardar los mapeos
mapping = {}

# Iteramos sobre las columnas y las transformamos
for col in cols:
    # Concatenamos los valores necesarios
    if col in ['Home', 'Away']:
        if 'Squad' not in mapping:
            name = 'Squad'
            squad = pd.concat([partidos['Home'], partidos['Away']])
            label_encoder.fit(squad)      
    elif col in ['Country (Home)', 'Country (Away)']:
        if 'Country' not in mapping:
            name = 'Country'
            country = pd.concat([partidos['Country (Home)'], partidos['Country (Away)']])
            label_encoder.fit(country)
    else:
        name = col
        label_encoder.fit(partidos[col])
    
    # Transformamos los valores 
    partidos[col] = label_encoder.transform(partidos[col])
    
    # Creamos un mapeo de los valores
    mapping[name] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Transformamos la columna 'Date' a datetime
partidos['Date'] = pd.to_datetime(partidos['Date'])

# Separar la fecha en año, mes y día
partidos['Year'] = partidos['Date'].dt.year
partidos['Month'] = partidos['Date'].dt.month
partidos['Number Day'] = partidos['Date'].dt.day # Lo llamamos 'Number Day' para evitar confusiones con la columna 'Day' que ya existe

# Eliminamos la columna 'Date'
partidos.drop('Date', axis=1, inplace=True)

# Verificamos los cambios
partidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 41 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Season          598 non-null    int32  
 1   Round           598 non-null    int32  
 2   Day             598 non-null    int32  
 3   Results         598 non-null    object 
 4   Home            598 non-null    int32  
 5   Country (Home)  598 non-null    int32  
 6   Points (Home)   598 non-null    float64
 7   Score (Home)    598 non-null    int64  
 8   Score (Away)    598 non-null    int64  
 9   Points (Away)   598 non-null    float64
 10  Country (Away)  598 non-null    int32  
 11  Away            598 non-null    int32  
 12  Venue           598 non-null    int32  
 13  Referee         598 non-null    int32  
 14  # Pl_home       540 non-null    float64
 15  Age_home        540 non-null    float64
 16  MP_home         540 non-null    float64
 17  Starts_home     540 non-null    flo

Eliminamos las filas con datos nulos.

In [30]:
partidos = partidos.dropna()

Ahora ya definimos nuestras variables para las predicciones. En nuestro caso, queremos predecir los resultados del partido, por lo que querremos predecir la columna 'Results'.

Eliminamos además las columnas 'Score (Home)' y 'Score (Away)' ya que en las predicciones nunca tendremos los resultados del partido, por lo que no tiene sentido entrenar los modelos con estas dos columnas. También eliminamos la columna 'Referee' ya que a la hora de predecir un partido tampoco sabremos quién será el árbitro del mismo.

In [31]:
X = partidos.drop(labels=['Results', 'Score (Home)', 'Score (Away)', 'Referee'], axis=1)
y = partidos['Results']

# Estandarizamos los datos

# Columnas a estandarizar
cols = [['Points (Home)', 'Points (Away)', '# Pl_home','Age_home','MP_home','Starts_home','Gls_home','Ast_home','G+A_home','G-PK_home','PK_home','PKatt_home','CrdY_home','CrdR_home','# Pl_away','Age_away','MP_away','Starts_away','Gls_away','Ast_away','G+A_away','G-PK_away','PK_away','PKatt_away','CrdY_away','CrdR_away']]

# Se recorren las columnas especificadas y se escala cada una
for col in cols:
    X[col] = scale(X[col])

Separamos en train y test las variables que usaremos para entrenar nuestros modelos.

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Modelos de predicción.

Comenzamos con la Regresión Logística.

In [33]:
lr = LogisticRegression(random_state=42, max_iter=10000, C=1.0)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

Seguimos con SVC (Support Vector Classification).

In [34]:
svc = SVC(random_state = 912, kernel='rbf')
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_test)

Nuestro último modelo de clasificación será XGBoost.

In [35]:
xgb = GradientBoostingClassifier(random_state=42)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

### Métricas de evaluación.

Para hacer estas métricas, haremos uso del informe de clasificación (Classification Report), una tabla que resume las métricas de evaluación del rendimiento de un modelo de clasificación en un conjunto de datos de prueba. 

¿Qué métricas aparecen en este informe?

- Precision (Precisión): La precisión indica la proporción de instancias clasificadas como positivas que son verdaderamente positivas. Se calcula como el número de verdaderos positivos dividido por el número total de predicciones positivas (verdaderos positivos más falsos positivos).

- Recall (Recuperación o Sensibilidad): El recall indica la proporción de instancias positivas que fueron correctamente clasificadas. Se calcula como el número de verdaderos positivos dividido por el número total de instancias positivas (verdaderos positivos más falsos negativos).

- F1-score (Puntaje F1): Es la media armónica de la precisión y el recall. Proporciona un equilibrio entre precisión y recall. Se calcula como 2 * (precision * recall) / (precision + recall).

- Support (Soporte): Es el número de muestras verdaderas que pertenecen a cada clase en el conjunto de datos de prueba.

- Accuracy (Exactitud): Es la proporción de muestras correctamente clasificadas entre todas las muestras. Se calcula como el número de muestras correctamente clasificadas dividido por el número total de muestras.

- Macro average (Promedio Macro): Es el promedio sin ponderar de las métricas para cada clase. Calcula las métricas independientemente para cada clase y luego toma el promedio sin tener en cuenta el desequilibrio en el tamaño de las clases.

- Weighted average (Promedio Ponderado): Es el promedio ponderado de las métricas para cada clase, donde cada clase contribuye con su peso relativo al número total de muestras en el conjunto de datos de prueba.

In [36]:
print("\nMétricas Regresión Logística:")
print("Precisión:", accuracy_score(y_test, lr_pred))
print("Informe de Clasificación:")
print(classification_report(y_test, lr_pred, zero_division=0))


Métricas Regresión Logística:
Precisión: 0.5679012345679012
Informe de Clasificación:
              precision    recall  f1-score   support

           A       0.57      0.43      0.49        54
           D       0.37      0.18      0.25        38
           H       0.60      0.89      0.72        70

    accuracy                           0.57       162
   macro avg       0.52      0.50      0.48       162
weighted avg       0.54      0.57      0.53       162



In [37]:
print("\nMétricas SVC:")
print("Precisión:", accuracy_score(y_test, svc_pred))
print("Informe de Clasificación:")
print(classification_report(y_test, svc_pred, zero_division=0))


Métricas SVC:
Precisión: 0.43209876543209874
Informe de Clasificación:
              precision    recall  f1-score   support

           A       0.00      0.00      0.00        54
           D       0.00      0.00      0.00        38
           H       0.43      1.00      0.60        70

    accuracy                           0.43       162
   macro avg       0.14      0.33      0.20       162
weighted avg       0.19      0.43      0.26       162



In [38]:
print("\nMétricas XGBoost:")
print("Precisión:", accuracy_score(y_test, xgb_pred))
print("Informe de Clasificación:")
print(classification_report(y_test, xgb_pred, zero_division=0))


Métricas XGBoost:
Precisión: 0.6049382716049383
Informe de Clasificación:
              precision    recall  f1-score   support

           A       0.68      0.52      0.59        54
           D       0.35      0.16      0.22        38
           H       0.62      0.91      0.74        70

    accuracy                           0.60       162
   macro avg       0.55      0.53      0.51       162
weighted avg       0.58      0.60      0.57       162



### Predicciones

Ahora hagamos las predicciones de los partidos que aún quedan por jugar.

#### Semifinales

In [39]:
# Definimos una lista llamada 'semis' que contiene datos de partidos de fútbol de las semifinales
semis = [['2023-2024', 'Semi-finals', 'Tue', 'Bayern Munich', 'Germany', 107.882298136646, 114.5545351473923, 'Spain', 'Real Madrid', 'Allianz Arena', 23, 28.3, 10, 110, 18, 14, 32, 16, 2, 2, 13.0, 1.0, 22, 28.0, 10, 110, 20, 17, 37, 20, 0, 1, 18.0, 0.0, 2024, 4, 30],
        ['2023-2024', 'Semi-finals', 'Wed', 'Dortmund', 'Germany', 91.17303312629399, 114.33458049886625, 'France', 'Paris S-G', 'Signal Iduna Park', 23, 28.0, 10, 110, 15, 12, 27, 14, 1, 1, 16.0, 0.0, 21, 25.3, 10, 110, 19, 12, 31, 16, 3, 3, 27.0, 0.0, 2024, 5, 1],
        ['2023-2024', 'Semi-finals', 'Tue', 'Paris S-G', 'France', 114.33458049886625, 91.17303312629399, 'Germany', 'Dortmund', 'Parc des Princes', 21, 25.3, 10, 110, 19, 12, 31, 16, 3, 3, 27.0, 0.0, 23, 28.0, 10, 110, 15, 12, 27, 14, 1, 1, 16.0, 0.0, 2024, 5, 7],
        ['2023-2024', 'Semi-finals', 'Wed', 'Real Madrid', 'Spain', 114.5545351473923, 107.882298136646, 'Germany', 'Bayern Munich', 'Estadio Santiago Bernabéu', 22, 28.0, 10, 110, 20, 17, 37, 20, 0, 1, 18.0, 0.0, 23, 28.3, 10, 110, 18, 14, 32, 16, 2, 2, 13.0, 1.0, 2024, 5, 8]]

# Obtenemos las columnas relevantes del DataFrame 'partidos' para usar como nombres de columnas en el DataFrame 'semis'
partidos_cols = partidos.drop(labels=['Results', 'Score (Home)', 'Score (Away)', 'Referee'], axis=1).columns

# Creamos un DataFrame 'semis' a partir de la lista 'semis' con las columnas obtenidas del DataFrame 'partidos'
semis = pd.DataFrame(semis, columns=(partidos_cols))

semis.head()

,Season,Round,Day,Home,Country (Home),Points (Home),Points (Away),Country (Away),Away,Venue,...,Ast_away,G+A_away,G-PK_away,PK_away,PKatt_away,CrdY_away,CrdR_away,Year,Month,Number Day
0,2023-2024,Semi-finals,Tue,Bayern Munich,Germany,107.882298,114.554535,Spain,Real Madrid,Allianz Arena,...,17,37,20,0,1,18.0,0.0,2024,4,30
1,2023-2024,Semi-finals,Wed,Dortmund,Germany,91.173033,114.334580,France,Paris S-G,Signal Iduna Park,...,12,31,16,3,3,27.0,0.0,2024,5,1
2,2023-2024,Semi-finals,Tue,Paris S-G,France,114.334580,91.173033,Germany,Dortmund,Parc des Princes,...,12,27,14,1,1,16.0,0.0,2024,5,7
3,2023-2024,Semi-finals,Wed,Real Madrid,Spain,114.554535,107.882298,Germany,Bayern Munich,Estadio Santiago Bernabéu,...,14,32,16,2,2,13.0,1.0,2024,5,8


Ahora, para poder aplicar nuestros modelos a estos nuevos datos, debemos convertirlos a numéricos. Para ello, aplicamos el mismo mapping que usamos anteriormente.

In [40]:
data = semis.copy()

# Aplicamos mapping a las columnas
for col, col_mapping in mapping.items():
    if col in data.columns:
        data[col] = data[col].map(col_mapping)
    else:
        if col == 'Squad':
            data['Home'] = data['Home'].map(col_mapping)
            data['Away'] = data['Away'].map(col_mapping)
        elif col == 'Country':
            data['Country (Home)'] = data['Country (Home)'].map(col_mapping)
            data['Country (Away)'] = data['Country (Away)'].map(col_mapping)

data.head()

,Season,Round,Day,Home,Country (Home),Points (Home),Points (Away),Country (Away),Away,Venue,...,Ast_away,G+A_away,G-PK_away,PK_away,PKatt_away,CrdY_away,CrdR_away,Year,Month,Number Day
0,20,3,4,7,7,107.882298,114.554535,13,50,1,...,17,37,20,0,1,18.0,0.0,2024,4,30
1,20,3,5,16,7,91.173033,114.334580,6,45,77,...,12,31,16,3,3,27.0,0.0,2024,5,1
2,20,3,4,45,6,114.334580,91.173033,7,16,71,...,12,27,14,1,1,16.0,0.0,2024,5,7
3,20,3,5,50,13,114.554535,107.882298,7,7,30,...,14,32,16,2,2,13.0,1.0,2024,5,8


Hacemos nuestras predicciones de las semifinales de este torneo

In [41]:
lr_pred_actual = lr.predict(data)
svc_pred_actual = svc.predict(data)
xgb_pred_actual = xgb.predict(data)

Para visualizar los resultados haremos un DataFrame nuevo. Para crear este nuevo DataFrame haremos 4 listas: cada una corresponde a una columna del nuevo DataFrame.

In [42]:
X_home = semis['Home'].tolist()
X_away = semis['Away'].tolist()
lr_pred_lst = lr_pred_actual.tolist()
svc_pred_lst = svc_pred_actual.tolist()
xgb_pred_lst = xgb_pred_actual.tolist()

# Se crea un DataFrame con los valores obtenidos
res = pd.DataFrame({'Home': X_home, 'Away': X_away, 'LR': lr_pred_lst, 'SVC': svc_pred_lst, 'XGB': xgb_pred_lst})

# Mostrar los resultados
res

,Home,Away,LR,SVC,XGB
0,Bayern Munich,Real Madrid,D,H,A
1,Dortmund,Paris S-G,D,H,A
2,Paris S-G,Dortmund,D,H,A
3,Real Madrid,Bayern Munich,D,H,A


No queda claro qué equipos pasarían a la siguiente ronda. Por lo tanto, estudiaremos las probabilidades que se han obtenido a la hora de hacer las predicciones. 

Es decir, el modelo hace una predicción porque, por ejemplo, la opción 'A' le ha salido con un 90% de probabilidad y las otras dos opciones con menor probabilidad. Basándose en estos datos, en la pantalla se muestra aquella opción con mayor probabilidad, en el caso que hemos visto, se monstraría 'A' como predicción. Nosotros analizaremos estas predicciones y el equipo que tenga mayor probabilidad de ganar pasa a la siguiente ronda.

In [43]:
# Predicción de probabilidades para Regresión Logística (lr)
lr_pred_prob = lr.predict_proba(data)

# Predicción de probabilidades para SVC (svc)
# Calcular las puntuaciones de decisión para SVC (svc)
svc_decision_scores = svc.decision_function(data)
# Convertir las puntuaciones en probabilidades usando la función sigmoide
svc_pred_prob = 1 / (1 + np.exp(-svc_decision_scores))

# Predicción de probabilidades para XGBoost (xgb)
xgb_pred_prob = xgb.predict_proba(data)

# Crear un diccionario con las probabilidades de cada modelo
pred_probs = {}

# Se obtienen las claves, que corresponden a los valores únicos de 'y' (variable objetivo)
pred_probs_keys = y.unique()

# Se recorren las claves y se obtienen las probabilidades de cada modelo
for key in pred_probs_keys:
    pred_probs[key] = []
    for i in range(len(lr_pred_prob)):
        # Asumimos que el orden de las clases 'y' es el mismo que el de sus predicciones
        # Se obtiene el índice de la clave en el array de claves
        x = np.where(pred_probs_keys == key)[0][0]
        # Se añaden las probabilidades de cada modelo a la lista de la clave correspondiente
        pred_probs[key].append([lr_pred_prob[i][x], svc_pred_prob[i][x], xgb_pred_prob[i][x]])

print(pred_probs)

{'A': [[1.3877743166022894e-39, 0.7313247032362016, 0.6248101347766639], [1.8702806329129012e-39, 0.7312892652543426, 0.5855941879747881], [4.486338985990076e-41, 0.731256194168103, 0.7821268305040091], [3.6776732833013395e-41, 0.7312873958966345, 0.6860468443149533]], 'H': [[1.0, 0.4446628147638641, 0.031781999126048066], [1.0, 0.4446612362739841, 0.025315575084039418], [1.0, 0.4446548777328631, 0.019438862409999483], [1.0, 0.44465083403174926, 0.04196448052392053]], 'D': [[8.710138864727451e-38, 0.9022173268404602, 0.3434078660972882], [4.413639654044017e-39, 0.9022196767409301, 0.3890902369411726], [8.291778537627145e-35, 0.9022236156425848, 0.19843430708599136], [1.1822212889580251e-36, 0.9022234956657642, 0.27198867516112607]]}


Una vez obtenidas las probabilidades, agregamos las más relevantes a nuestra tabla de resultados.

In [44]:
# Iterar sobre las filas y columnas del DataFrame 'res'
for index, row in res.iterrows():
    for col in ['LR', 'SVC', 'XGB']:  # Iterar sobre las columnas LR, SVC y XGB
        col_index = ['LR', 'SVC', 'XGB'].index(col)
        # Obtener el valor de 'key' basado en el valor en la columna 'col'
        key = row[col]
        # Obtener el valor asociado de 'pred_probs' para la fila 'index' y columna 'key'
        value = pred_probs[key][index]
        value = value[col_index]
        # Actualizar la celda en el DataFrame con el valor 'L: value'
        res.at[index, col] = f"{key}: {value}"

# Mostrar los resultados actualizados
res

,Home,Away,LR,SVC,XGB
0,Bayern Munich,Real Madrid,D: 8.710138864727451e-38,H: 0.4446628147638641,A: 0.6248101347766639
1,Dortmund,Paris S-G,D: 4.413639654044017e-39,H: 0.4446612362739841,A: 0.5855941879747881
2,Paris S-G,Dortmund,D: 8.291778537627145e-35,H: 0.4446548777328631,A: 0.7821268305040091
3,Real Madrid,Bayern Munich,D: 1.1822212889580251e-36,H: 0.44465083403174926,A: 0.6860468443149533


Vemos que para nuestro modelo de XGBoost son claros ganadores el Bayern Munich y Dortmund. 

Por otro lado, en SVC las probabilidades están más igualdas pero el resultado es el mismo: pasan a la final el Bayern Munich y Dortmund.

Por último, en la Regresión Logística no está tan claro el resultado de los partidos. Analicemos más a fondo los resultados obtenidos por nuestro modelo de Regresión Logística.

In [45]:
print('Bayern Munich vs Real Madrid:')
print('H: ', pred_probs['H'][0][0])
print('D: ', pred_probs['D'][0][0])
print('A: ', pred_probs['A'][0][0])

print('-' * 50)

print('Dortmund vs Paris S-G:')
print('H: ', pred_probs['H'][1][0])
print('D: ', pred_probs['D'][1][0])
print('A: ', pred_probs['A'][1][0])

print('-' * 50)

print('Paris S-G vs Dortmund:')
print('H: ', pred_probs['H'][2][0])
print('D: ', pred_probs['D'][2][0])
print('A: ', pred_probs['A'][2][0])

print('-' * 50)

print('Real Madrid vs Bayern Munich:')
print('H: ', pred_probs['H'][3][0])
print('D: ', pred_probs['D'][3][0])
print('A: ', pred_probs['A'][3][0])

Bayern Munich vs Real Madrid:
H:  1.0
D:  8.710138864727451e-38
A:  1.3877743166022894e-39
--------------------------------------------------
Dortmund vs Paris S-G:
H:  1.0
D:  4.413639654044017e-39
A:  1.8702806329129012e-39
--------------------------------------------------
Paris S-G vs Dortmund:
H:  1.0
D:  8.291778537627145e-35
A:  4.486338985990076e-41
--------------------------------------------------
Real Madrid vs Bayern Munich:
H:  1.0
D:  1.1822212889580251e-36
A:  3.6776732833013395e-41


Podemos ver que el porcentaje de que gane el equipo de casa es extemandamente alto, casi 1, mientras que el resto de opciones tienen porcentajes casi nulos. Para determinar quién pasa a la final, veremos quién pierde con menos probabilidad. Vemos que la probabilidad de que gane el Real Madrid en el primer partido es más grande que la probabilidad de que el Bayern en el último. Por lo tanto, de estos dos equipos pasaría el Real Madrid.

Por otro lado, pasa lo mismo con el Dortmund y el PSG, la probabilidad de que gane el segundo partido el PSG es mayor que la probabilidad de que gane el tercero el Dortmund. Por lo tanro, pasaría a la final el PSG.


Por lo tanto, concluimos que nuestros modelo de clasificación nos dan dos finales distintas:
1º Real Madrid vs PSG.
2º Bayern Munich vs Dortmund.

#### Final

Creamos DataFrames de las dos posibles finales.

In [46]:
final1 = [['2023-2024', 'Final', 'Sat', 'Real Madrid', 'Spain', 114.5545351473923, 114.33458049886625, 'France', 'Paris S-G', 'Wembley Stadium', 22, 28.0, 10, 110, 20, 17, 37, 20, 0, 1, 18.0, 0.0, 21, 25.3, 10, 110, 19, 12, 31, 16, 3, 3, 27.0, 0.0, 2024, 6, 1]]
final1 = pd.DataFrame(final1, columns=(partidos_cols))

final1.head()

,Season,Round,Day,Home,Country (Home),Points (Home),Points (Away),Country (Away),Away,Venue,...,Ast_away,G+A_away,G-PK_away,PK_away,PKatt_away,CrdY_away,CrdR_away,Year,Month,Number Day
0,2023-2024,Final,Sat,Real Madrid,Spain,114.554535,114.33458,France,Paris S-G,Wembley Stadium,...,12,31,16,3,3,27.0,0.0,2024,6,1


In [47]:
final2 = [['2023-2024', 'Final', 'Sat', 'Bayern Munich', 'Germany', 107.882298136646, 91.17303312629399, 'Germany', 'Dortmund', 'Wembley Stadium', 23,28.3,10,110,18,14,32,16,2,2,13.0,1.0, 23,28.0,10,110,15,12,27,14,1,1,16.0,0.0, 2024, 6, 1]]
final2 = pd.DataFrame(final2, columns=(partidos_cols))

final2.head()

,Season,Round,Day,Home,Country (Home),Points (Home),Points (Away),Country (Away),Away,Venue,...,Ast_away,G+A_away,G-PK_away,PK_away,PKatt_away,CrdY_away,CrdR_away,Year,Month,Number Day
0,2023-2024,Final,Sat,Bayern Munich,Germany,107.882298,91.173033,Germany,Dortmund,Wembley Stadium,...,12,27,14,1,1,16.0,0.0,2024,6,1


Generamos dos sets de datos dependiendo de la final jugada y tranformamos las columnas a numéricas usando el mismo mapping.

In [48]:
data1 = final1.copy()

# Aplicamos mapping a las columnas
for col, col_mapping in mapping.items():
    if col in data1.columns:
        data1[col] = data1[col].map(col_mapping)
    else:
        if col == 'Squad':
            data1['Home'] = data1['Home'].map(col_mapping)
            data1['Away'] = data1['Away'].map(col_mapping)
        elif col == 'Country':
            data1['Country (Home)'] = data1['Country (Home)'].map(col_mapping)
            data1['Country (Away)'] = data1['Country (Away)'].map(col_mapping)

data1.head()

,Season,Round,Day,Home,Country (Home),Points (Home),Points (Away),Country (Away),Away,Venue,...,Ast_away,G+A_away,G-PK_away,PK_away,PKatt_away,CrdY_away,CrdR_away,Year,Month,Number Day
0,20,0,1,50,13,114.554535,114.33458,6,45,111,...,12,31,16,3,3,27.0,0.0,2024,6,1


In [49]:
data2 = final2.copy()

# Aplicamos mapping a las columnas
for col, col_mapping in mapping.items():
    if col in data2.columns:
        data2[col] = data2[col].map(col_mapping)
    else:
        if col == 'Squad':
            data2['Home'] = data2['Home'].map(col_mapping)
            data2['Away'] = data2['Away'].map(col_mapping)
        elif col == 'Country':
            data2['Country (Home)'] = data2['Country (Home)'].map(col_mapping)
            data2['Country (Away)'] = data2['Country (Away)'].map(col_mapping)

data2.head()

,Season,Round,Day,Home,Country (Home),Points (Home),Points (Away),Country (Away),Away,Venue,...,Ast_away,G+A_away,G-PK_away,PK_away,PKatt_away,CrdY_away,CrdR_away,Year,Month,Number Day
0,20,0,1,7,7,107.882298,91.173033,7,16,111,...,12,27,14,1,1,16.0,0.0,2024,6,1


Predecimos el resultado de la final con nuestros modelos.

In [50]:
lr_pred_actual = lr.predict(data1)
svc_pred_actual = svc.predict(data2)
xgb_pred_actual = xgb.predict(data2)

Visualizamos y analizamos los resultados.

In [51]:
X_home = []
X_away = []
preds = []
pred_vals = [lr_pred_actual[0], svc_pred_actual[0], xgb_pred_actual[0]]

for i in range(3): 
    if i == 0:
        h = final1['Home'][0]
        a = final1['Away'][0]
        pred = 'LR'
    else:
        h = final2['Home'][0]
        a = final2['Away'][0]
        if i == 1:
            pred = 'SVC'
        elif i == 2:
            pred = 'XGB'

    X_home.append(h)
    X_away.append(a)
    preds.append(pred)

# Se crea un DataFrame con los valores obtenidos
res = pd.DataFrame({'Home': X_home, 'Away': X_away, 'Prediction': preds, 'Value': pred_vals})

# Mostrar los resultados
res

,Home,Away,Prediction,Value
0,Real Madrid,Paris S-G,LR,D
1,Bayern Munich,Dortmund,SVC,H
2,Bayern Munich,Dortmund,XGB,A


De nuevo, La regresión Logística no nos da resultados concluyentes, analicemos más a fondo.

In [52]:
# Predicción de probabilidades para Regresión Logística (lr)
lr_pred_prob = lr.predict_proba(data1)

print('Resultados Regresión Logística:')
print('A: ', lr_pred_prob[0][0])
print('H: ', lr_pred_prob[0][1])
print('D: ', lr_pred_prob[0][2])

Resultados Regresión Logística:
A:  5.440460854561761e-40
H:  1.0
D:  6.918676551640068e-36


Finalmente podemos concluir que gana el Bayern Munich en el modelo SVC, el Real Madrid gana en el modelo de Regresión Logística y el Dortmund en XGBoost.